## Step 1: Creating Tokens

In [1]:
with open('the-verdict.txt', 'r', encoding = 'utf-8') as f:
    raw_text = f.read()

print('Total number of characters: ', len(raw_text))
print(raw_text[:99])

Total number of characters:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [2]:
import re

text = 'Hello, world. This, is a test.'
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [3]:
result = re.split(r'([,.]|\s)', text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [4]:
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [5]:
text = 'Hello, world. Is this-- a test?'

result = re.split(r'([,.:;?_!"()\']|--|\s)', text) # Splitting the sentence wherever is the white space or any of the symbols ([,.:;?_!"()\']|--|\s)
result = [item.strip() for item in result if item.strip()] # Removing white spaces

print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [6]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [7]:
print(len(preprocessed))

4690


## Step 2: Creating Token IDs

In [8]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [9]:
vocab = {token:integer for integer, token in enumerate(all_words)}

In [10]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [11]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()} # Take every string (token) and integer (token id) from the vocabulary and flip it

    def encode(self, text):    # Takes tokens from prepreprocessed and converts them into id-s
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

        preprocessed = [
            item.strip() for item in preprocessed if item.strip() 
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids    

    def decode(self, ids):    # Converting token id-s into tokens
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified functions
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [12]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
           Mr's Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 66, 2, 850, 38, 851, 1108, 754, 793, 7]


In [13]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mr\' s Gisburn said with pardonable pride.'

In [14]:
text = 'Hello, Do you like tea?'
print(tokenizer.encode(text))

KeyError: 'Hello'

##### Word 'Hello' is not into our vocabulary

---

<div class='alert alert-block alert-info'>
    
## Adding special context tokens

The purpose is to modify this tokenizer to handle unknown words.

    <|unk|> : Stores unknown tokens. Example: The fox chased dog
                                               0   1     2    3

                                              The fox chased dog quickly
                                               0   1     2    3     783*

    <|endoftext|> : When we have multiple text sources, on example 3 books

</div>

In [15]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(['<|endoftext|>', '<|unk|>'])

vocab = {token:integer for integer, token in enumerate(all_tokens)}

In [16]:
len(vocab.items())

1132

In [17]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [18]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()} # Take every string (token) and integer (token id) from the vocabulary and flip it

    def encode(self, text):    # Takes tokens from prepreprocessed and converts them into id-s
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip() 
        ]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids    

    def decode(self, ids):    # Converting token id-s into tokens
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified functions
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [19]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = 'Hello, do you like tea?'
text2 = 'In the sunlit terraces of the palace.'

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [20]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [21]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

In [22]:
! pip3 install tiktoken 

   ---------------------------------------- 0.0/893.9 kB ? eta -:--:--
   ---------------------------------------- 10.2/893.9 kB ? eta -:--:--
   -- ------------------------------------ 61.4/893.9 kB 812.7 kB/s eta 0:00:02
   --------------------- ------------------ 471.0/893.9 kB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 893.9/893.9 kB 5.7 MB/s eta 0:00:00


In [23]:
import importlib
import tiktoken

print('tiktoken version: ', importlib.metadata.version('tiktoken'))

tiktoken version:  0.9.0


In [24]:
tokenizer = tiktoken.get_encoding('gpt2')

In [25]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special = {"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [26]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [27]:
integers = tokenizer.encode("Akwirw ier")
print(integers)

strings = tokenizer.decode(integers)
print(strings)

[33901, 86, 343, 86, 220, 959]
Akwirw ier
